In [5]:
__author__ = "Marcus Heide"
__version__ = "1.2"
__email__ = "marcus.heide@b-tu.de"
__status__ = "Test"

from ipywidgets import interact, interactive, fixed, interact_manual, Layout
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft
from scipy.signal import find_peaks
'exec(%matplotlib inline)' #%matplotlib widget
#%matplotlib inline # Alternative zu widget für statische Plots 

linewidth = 2
fontsize = 11
#figsize = (7, 4.5)

from IPython.display import set_matplotlib_formats #, Math, Latex
set_matplotlib_formats('pdf', 'png')

plt.rcParams['savefig.dpi'] = 75
plt.rcParams['figure.autolayout'] = False
plt.rcParams['figure.figsize'] = 7, 4.5
plt.rcParams['axes.labelsize'] = 16
plt.rcParams['axes.titlesize'] = 18
plt.rcParams['font.size'] = 16
plt.rcParams['lines.linewidth'] = 2.0
plt.rcParams['lines.markersize'] = 8
plt.rcParams['legend.fontsize'] = 14
plt.rcParams['font.family'] = "STIXGeneral"
plt.rcParams['mathtext.fontset'] = 'stix'

 # Demonstrationsparameter
f_1 = 1 # .0e9 # 24.0 GHz
B = 100 # e6 # 250 MHz
T_sweep = 1 # 000e-6 # 1000us
targets_txt = '0.5/0.05 0.5/0.07 0.0001/0.12'
peak_threshold = 0.01
peak_distance = 5
Nfreq = 100
Nwave = 5000
zeropadfac = 8

 # realistische Parameter - ungünstige Darstellung
# f_1 = 24e9 # .0e9 # 24.0 GHz
# B = 250e6 # e6 # 250 MHz
# T_sweep = 1e-3 # 000e-6 # 1000us
# alpha = 0.5
# targets = 33e-9
# Ncorse = 500
# Nfine = 5000

def is_float(value):
    try:
        float(value)
        return True
    except:
        return False

 # Berechnung des gesendeten Sweeps - zeitlicher Frequenzverlauf
def calc_tx_frq(t, f_1, B, T_sweep):
    y = f_1 + B / T_sweep * t * (t <= T_sweep)
    return y

 # Berechnung der empfangenen Sweeps - zeitlicher Frequenzverlauf
def calc_rx_frq(t, f_1, B, T_sweep, alpha, t_d):
    y = f_1 + B / T_sweep * (t - t_d) * (t >= t_d) * (t <= (t_d + T_sweep))
    return y

 # Berechnung des gesendeten Sweeps - zeitlicher Amplitudenverlauf
def calc_tx_wave(t, f_1, B, T_sweep):
    y = np.cos(2 * np.pi * f_1 * t + np.pi * B / T_sweep * t**2) * (t <= T_sweep)
    return y

 # Berechnung der empfangenen Sweeps - zeitlicher Amplitudenverlauf
def calc_rx_wave(t, f_1, B, T_sweep, alpha, t_d):
    y = alpha * np.cos(2 * np.pi * f_1 * (t - t_d) + np.pi * B / T_sweep * (t - t_d)**2) * (t >= t_d)
    return y

 # Berechnung des Mischsignals - zeitlicher Amplitudenverlauf
def calc_mix_wave(t, f_1, B, T_sweep, alpha, t_d):  
    y = np.cos(2 * np.pi * f_1 * t + np.pi * B / T_sweep * t**2) * alpha * np.cos(2 * np.pi * f_1 * (t - t_d) + np.pi * B / T_sweep * (t - t_d)**2) * (t >= t_d) * (t <= (t_d + T_sweep))
    return y

 # Berechnung des IF-Signals - zeitlicher Amplitudenverlauf
def calc_zf_wave(t, f_1, B, T_sweep, alpha, t_d):  
    y = 1/2 * alpha * np.cos(-2*np.pi * f_1 * (t_d) + np.pi * B / T_sweep * t_d ** 2 - 2 * np.pi * B / T_sweep * t_d * t) * (t >= t_d) * (t <= (t_d + T_sweep))
    return y

 # Berechnung der Fast Fourier Transformation
def calc_fft(t, y):
    N = y.size*zeropadfac
    t_s = t[1]-t[0]
    f = np.linspace(0, 1 / (2 * t_s), int(N / 2))
    y_win = y * np.hanning(y.size)
    yf = fft(y_win, n = N)
    yf = 2/N * np.abs(yf[:N//2])
    yf[0] /= 2
    return f, yf

targets = targets_txt.split(' ')
alphas = []
t_ds = []
for target in targets:
    alpha, t_d = target.split('/')
    if is_float(alpha):
        alphas.append(float(alpha))
    if is_float(t_d):
        t_ds.append(float(t_d))

 # Zeitachsen generieren - 2 Stück mit unterschiedlicher Auflösung
t_f = np.linspace(0, T_sweep + float(max(t_ds)), Nfreq)
t_f = np.append(t_f, [t_f[-1]+t_f[1], t_f[-1]+2*t_f[1]]) # für eine besseren Darstellung werden noch 2 Samples angehangen
t_w = np.linspace(0, T_sweep + float(max(t_ds)), Nwave)

 # Frequenzzeitverläufe
out0 = widgets.Output()
with out0:
    fig0, ax0 = plt.subplots(1, 1)#, figsize = figsize);
    yf_t = calc_tx_frq(t_f, f_1, B, T_sweep)
    ax0.plot(t_f, yf_t, label = 'Sendesignal', lw = linewidth)
    
    ix = 0
    for alpha,t_d in zip(alphas,t_ds):
        ix += 1
        yf_r = calc_rx_frq(t_f, f_1, B, T_sweep, alpha, t_d)
        ax0.plot(t_f, yf_r, color = '#ff7f0e', label = r'{}. Empfangssignal $\alpha$ = {} $\Delta t$ = {} s'.format(ix,alpha,t_d), lw = linewidth)

 # Amplitudenzeitverläufe
out1 = widgets.Output()
with out1:
    fig1, ax1 = plt.subplots(1, 1)#, figsize = figsize);
    y_tx = calc_tx_wave(t_w, f_1, B, T_sweep)
    line_tx, = ax1.plot(t_w, y_tx, label = 'Sendesignal', lw = linewidth)
    y_rx = [0] * len(t_w)
    y_mix = [0] * len(t_w)
    y_if = [0] * len(t_w)
    for alpha,t_d in zip(alphas,t_ds):
        y_rx += calc_rx_wave(t_w, f_1, B, T_sweep, alpha, t_d)
        y_mix += calc_mix_wave(t_w, f_1, B, T_sweep, alpha, t_d)
        y_if += calc_zf_wave(t_w, f_1, B, T_sweep, alpha, t_d)

line_rx, = ax1.plot(t_w, y_rx, label = '$\sum$ Empfangssignal', lw = linewidth)
line_mix, = ax1.plot(t_w, y_mix, label = 'Mischsignal', lw = linewidth)
line_if, = ax1.plot(t_w, y_if, label = 'IF-Signal', lw = linewidth)

 # Fourieranalyse
out2 = widgets.Output()
with out2:
    fig2, ax2 = plt.subplots(1, 1)#, figsize = figsize)
    idx = (np.abs(t_w - T_sweep)).argmin()
    f, yf = calc_fft(t_w[:idx], y_if[:idx])
    line_fft, = ax2.plot(f, yf, ".", markersize = 6, linestyle = 'solid', color= '#d62728', label = r'IF-Signal $f_{{0}}$ = ~{} Hz'.format(1/T_sweep), lw = linewidth)
    #peaks, _ = find_peaks(yf, threshold = peak_threshold, distance = peak_distance)
    #line_peaks, = ax2.plot(f[peaks], yf[peaks], "x", markeredgewidth = 3, markersize = 12, color = '#ff7f0e')
    #line_peaks.set_label('Maxima {} Hz'.format(np.round(f[peaks],1)))

 # Styling der Plot Figures
def style_figs():
    figs = list(map(plt.figure, plt.get_fignums()))
    for fig in figs:
        fig.canvas.toolbar_position = 'left'
        fig.canvas.header_visible = False
        fig.canvas.footer_visible = False
        fig.canvas.resizable = False
        for ax in fig.axes: # alle Achsen durchgehen
            ax.spines['right'].set_color('none')
            ax.spines['top'].set_color('none')
            ax.ticklabel_format(axis = 'both', style = 'sci', useMathText=True) #scilimits = (0,0), 

             # Frequenz-Zeit-Diagramm Figure
            if fig == figs[0]:
                ax.set_xlim(0,ax.get_xlim()[1])
                ax.set_ylim(0,ax.get_ylim()[1])
                ax.set_title('Frequenz-Zeit-Diagramm')
                ax.set_xlabel('Zeit [s]')
                ax.set_ylabel('Frequenz')
                ax.spines['left'].set_position(('data', 0))
                ax.spines['bottom'].set_position(('data', 0))
                if wid_dict['tx_checkbox'].value or wid_dict['rx_checkbox'].value:
                    ax.legend(bbox_to_anchor=(0.05, 1), loc='upper left')
                else:
                    ax.legend_ = None

             # Amplituden-Zeit-Diagramm Figure
            if fig == figs[1]:
                ax.set_xlim(0,t_w[-1])
                #ax.set_xlim(0,ax.lines[0].get_xdata()[-1])
                #ax.set_ylim(0,ax.get_ylim()[1])
                ax.set_title('Amplituden-Zeit-Diagramm')
                ax.set_xlabel('Zeit [s]')
                ax.set_ylabel('Normierte Amplitude')
                ax.spines['bottom'].set_position(('data',0))
                ax.yaxis.set_ticks_position('left')
                ax.spines['left'].set_position(('data',0))
                ax.spines['bottom'].set_position(('axes', 0))
                if wid_dict['tx_checkbox'].value or wid_dict['rx_checkbox'].value or wid_dict['mix_checkbox'].value or wid_dict['if_checkbox'].value:
                    ax.legend(loc='lower right')
                else:
                    ax.legend_ = None

             # Fourieranalyse Figure
            if fig == figs[2]:
                ax.set_xlim(0, max(t_ds) * B / T_sweep) # FFT Darstellung auf die höchste Frequenz zoomen 
                ax.title.set_text('Frequenzspektrum')
                ax.set_xlabel('Frequenz [Hz]')
                ax.set_ylabel('Normierte Amplitude')
                ax.spines['left'].set_position(('data', 0))
                ax.spines['bottom'].set_position(('data', 0))
                if wid_dict['if_checkbox'].value:
                    ax.legend(loc='upper right')
                else:
                    ax.legend_ = None
            
            ax.relim()
            ax.autoscale_view(True,True,True)
            fig.tight_layout()

#style_figs()

 # Kurven neuberechnen, Plots aktualisieren
def update_plots():
    global f_1, B, T_sweep, targets
    targets = targets.split(' ')
    global alphas, t_ds
    alphas = []
    t_ds = []
    for target in targets:
        if not target.find('/') == -1:
            alpha, t_d = target.split('/')
            if is_float(alpha):
                alphas.append(float(alpha))
            if is_float(t_d):
                t_ds.append(float(t_d))

    global t_f, t_w
    t_f = np.linspace(0, T_sweep + float(max(t_ds)), Nfreq)
    t_f = np.append(t_f, [t_f[-1]+t_f[1], t_f[-1]+2*t_f[1]]) # für eine besseren Darstellung werden noch 2 Samples angehangen
    t_w = np.linspace(0, T_sweep + float(max(t_ds)), Nwave)

    ax0.clear()
    if wid_dict['tx_checkbox'].value:
        global yf_t
        yf_t = calc_tx_frq(t_f, f_1, B, T_sweep)
        ax0.plot(t_f, yf_t, label = 'Sendesignal', lw = linewidth)
    else:
        ax0.plot([], [], label = '', lw = linewidth)

    ix = 0
    if wid_dict['rx_checkbox'].value:
        for alpha,t_d in zip(alphas,t_ds):
            ix += 1
            yf_r = calc_rx_frq(t_f, f_1, B, T_sweep, alpha, t_d)
            ax0.plot(t_f, yf_r, color = '#ff7f0e', label = r'{}. Empfangssignal $\alpha$ = {} $\Delta t$ = {} s'.format(ix,alpha,t_d), lw = linewidth)
    ax0.set_title('Frequenz-Zeit-Diagramm')
    ax0.set_xlabel('Zeit [s]')
    ax0.set_ylabel('Frequenz')
    fig0.tight_layout()

    if wid_dict['tx_checkbox'].value:
        global y_tx
        y_tx = calc_tx_wave(t_w, f_1, B, T_sweep)
        line_tx.set_xdata(t_w)
        line_tx.set_ydata(y_tx)
        line_tx.set_label('Sendesignal')
    else:
        line_tx.set_xdata([])
        line_tx.set_ydata([])
        line_tx.set_label('')

    if wid_dict['rx_checkbox'].value:
        global y_rx
        y_rx = [0] * len(t_w)
        for alpha,t_d in zip(alphas,t_ds):
            y_rx += calc_rx_wave(t_w, f_1, B, T_sweep, alpha, t_d)
        line_rx.set_xdata(t_w)
        line_rx.set_ydata(y_rx)
        line_rx.set_label('$\sum$ Empfangssignal')
    else:
        line_rx.set_xdata([])
        line_rx.set_ydata([])
        line_rx.set_label('')

    if wid_dict['mix_checkbox'].value:
        global y_mix
        y_mix = [0] * len(t_w)
        for alpha,t_d in zip(alphas,t_ds):
            y_mix += calc_mix_wave(t_w, f_1, B, T_sweep, alpha, t_d)
        line_mix.set_xdata(t_w)
        line_mix.set_ydata(y_mix)
        line_mix.set_label('Mischsignal')
    else:
        line_mix.set_xdata([])
        line_mix.set_ydata([])
        line_mix.set_label('')

    if wid_dict['if_checkbox'].value:
        global y_if
        y_if = [0] * len(t_w)
        for alpha,t_d in zip(alphas,t_ds):
            y_if += calc_zf_wave(t_w, f_1, B, T_sweep, alpha, t_d)
        line_if.set_xdata(t_w)
        line_if.set_ydata(y_if)
        line_if.set_label('IF-Signal')
        idx = (np.abs(t_w - T_sweep)).argmin()
        global f, yf
        f, yf = calc_fft(t_w[:idx], y_if[:idx])
        line_fft.set_xdata(f)
        line_fft.set_ydata(yf)
        line_fft.set_label('IF-Signal $f_{{0}}$ = ~{} Hz'.format(1/T_sweep))
        #global peaks
        #peaks, _ = find_peaks(yf, threshold = peak_threshold)
        #line_peaks.set_xdata(f[peaks])
        #line_peaks.set_ydata(yf[peaks])
        #line_peaks.set_label('Maxima {} Hz'.format(np.round(f[peaks],1)))
    else:
        line_if.set_xdata([])
        line_if.set_ydata([])
        line_if.set_label('')
        line_fft.set_xdata([])
        line_fft.set_ydata([])
        line_fft.set_label('')
        line_peaks.set_xdata([])
        line_peaks.set_ydata([])
        line_peaks.set_label('')
    style_figs()

 # Parameterwechsel verarbeiten und darstellen 
def on_change(change):       
    if (is_float(wid_dict['f_1_input'].value) and is_float(wid_dict['B_input'].value) and is_float(wid_dict['T_sweep_input'].value) and
        float(wid_dict['T_sweep_input'].value) > 0):
        global f_1, B, T_sweep, targets
        f_1 = float(wid_dict['f_1_input'].value)
        B = float(wid_dict['B_input'].value)
        T_sweep = float(wid_dict['T_sweep_input'].value)
        targets = wid_dict['targets_input'].value
        update_plots() 

 # Bedieneleemente erstellen
controls_params = widgets.VBox() # Horizontale Box zum Gruppieren der FMCW-Paramter Bedienelemente
controls_visibility = widgets.VBox() # Horizontale Box zum Gruppieren der Plot Sichtbarkeitsbedienelemente
wid_dict = {}
wid_name = ['f_1_input', 'B_input', 'T_sweep_input', 'targets_input', 'tx_checkbox', 'rx_checkbox', 'mix_checkbox', 'if_checkbox']
wid_value = [f_1, B, T_sweep, targets_txt, True, True, True, True]
wid_desc = ['Frequenz [Hz]', 'Bandbreite [Hz]', 'Sweepdauer [s]', 'Ziele[Dämpfung/Laufzeit [s]]', 'Sendesignal', 'Empfangssignal', 'Mischsignal', 'IF-Signal']
wid_width = ['180px', '180px', '180px', '500px', '100px', '120px', '100px', '180px']
for wid_name, wid_value, wid_desc, wid_width in zip(wid_name, wid_value, wid_desc, wid_width):
    if wid_name[-6:] == '_input':
        wid_dict[wid_name] = widgets.Text(str(wid_value), description = wid_desc, style = {'description_width': 'initial'}, layout = Layout(width = wid_width))
        controls_params.children += (wid_dict[wid_name],)
    elif wid_name[-9:] == '_checkbox':
        wid_dict[wid_name] = widgets.Checkbox(wid_value, description = wid_desc, disabled = False, indent = False, style = {'description_width': 'initial'}, layout = Layout(width = wid_width))
        controls_visibility.children += (wid_dict[wid_name],)

style_figs()

 # Aktualisierungsbutton
def on_refesh_button_clicked(b):
    if (is_float(wid_dict['f_1_input'].value) and is_float(wid_dict['B_input'].value) and is_float(wid_dict['T_sweep_input'].value) and
        float(wid_dict['T_sweep_input'].value) > 0):
        global f_1, B, T_sweep, targets
        f_1 = float(wid_dict['f_1_input'].value)
        B = float(wid_dict['B_input'].value)
        T_sweep = float(wid_dict['T_sweep_input'].value)
        targets = wid_dict['targets_input'].value
        update_plots()

 # Anordnung der Widgets
refresh_button = widgets.Button(description="Aktualisieren")
controls = widgets.VBox([widgets.Label(value = 'FMCW Parameter'), controls_params, controls_visibility, refresh_button], layout = Layout(margin='80px 50px 50px 150px'))#, border='dashed 1px'))
out0_controls_params = widgets.HBox([out0,controls])

title_label = widgets.HTML(value = '<p style="font-family:georgia,garamond,serif; font-size:24px; text-align: center;"><b>FMCW-Radar-Simulator für statische Ziele</b></p>',
                           layout = Layout(margin='50px 0px 0px 0px'))
display(title_label)
display(out0_controls_params)
out0_out2 = widgets.HBox([out1,out2])
display(out0_out2)

refresh_button.on_click(on_refesh_button_clicked)

HTML(value='<p style="font-family:georgia,garamond,serif; font-size:24px; text-align: center;"><b>FMCW-Radar-S…